- 전체 문제에 대해 실행

In [21]:
import pandas as pd
import time
import os
import re
import openai
from dotenv import load_dotenv
from openai import OpenAI

# 환경 설정
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 엑셀 파일 로드
df = pd.read_excel("수학문제생성_20문제_채점기준.xlsx")

# 결과 저장 리스트
results = []

def generate_and_score_all():
    for idx, row in df.iterrows():
        question_id = row['question_id']
        question = row['question_text']
        solution = row['solution_text']
        answer = row['final_answer']
        rubric = row['rubric']

        prompt_generate = f"""다음 중학교 수학 문제에 대해 총 5개의 서술형 답안을 생성해줘.

- 단답형이 아니라 서술형 문제에 대한 답안이야.
- 앞의 2개는 수학 전문가처럼 정확하고 자세한 풀이로 작성해줘.
- 뒤의 3개는 정답이 아예 틀린 풀이야. 표현은 실제 학생처럼 자신감 있게, 개념 오해나 계산 실수 포함.
- 대화체나 추측 표현은 사용하지 마.
- 명확하고 논리적인 문장을 사용해줘.
- 풀이 도입부에 "이 문제를 해결하기 위해", "소수를 찾기 위해" 같은 설명 문장은 생략해줘.
- 답안은 생각의 과정을 설명하는 글이 아니야. 풀이 과정에서 무엇을 검토해야 하는지, 어떤 생각을 했는지 말하지 마.
- "확인해보자", "검토해보자", "생각해보면" 같은 문장은 쓰지 마.
- 답안은 이미 검토를 마치고 작성한 문장처럼 서술해줘.

[출력 형식]
답안은 아래와 같은 형식으로 출력해줘:

답안 1. ...
답안 2. ...
답안 3. ...
답안 4. ...
답안 5. ...

각 답안 앞에 반드시 '답안 1.', '답안 2.' 와 같은 형식을 붙여줘. 숫자와 마침표는 포함해줘.

[문제]
{question}

[정답]
{answer}

[모범 풀이]
{solution}

[5개의 학생 답안]"""

        try:
            response_generate = client.chat.completions.create(
                model="gpt-4-0125-preview",
                messages=[{"role": "user", "content": prompt_generate}],
                temperature=0.8,
                max_tokens=1200
            )

            answer_text = response_generate.choices[0].message.content.strip()
            split_answers = re.split(r"\n?\s*답안\s*[1-5]\.\s*", answer_text)
            student_answers = [a.strip() for a in split_answers if a.strip()][0:5]

            for i, student_answer in enumerate(student_answers, start=1):
                prompt_score = f"""너는 중학교 수학 교사야. 아래 학생의 서술형 답안을 채점 기준(rubric)에 따라 평가해줘.

[문제]
{question}

[채점 기준]
{rubric}

[학생 답안]
{student_answer}

[목표]
- 각 rubric 항목에 대해 학생이 해당 기준을 충족했는지를 분석해줘.
- 기준을 충족했으면 구체적으로 어떤 점에서 잘했는지, 충족하지 못했으면 어떤 오류나 개념 부족이 있었는지 구체적으로 설명해줘.
- 마지막엔 정답인지, 오답인지를 아래와 같은 형식으로 명확히 판단해줘.

[문체 지시]
- 존댓말은 쓰지 말고, '~다'체의 설명문으로 작성해줘. (예: ~했다, ~이다, ~아니다)
- "이 항목도 충족합니다" 같은 반복적이고 불필요한 문장은 쓰지 말아줘.
- 각 항목에 대해 핵심적인 채점 사유만 간결하게 정리해줘.

[출력 형식 예시]

1. (rubric 기준 1)
- 설명

2. (rubric 기준 2)
- 설명

...

따라서 학생의 답안은 "정답" 또는 "오답"
"""

                response_score = client.chat.completions.create(
                    model="gpt-4-0125-preview",
                    messages=[{"role": "user", "content": prompt_score}],
                    temperature=0.5,
                    max_tokens=1000
                )

                grading_result = response_score.choices[0].message.content.strip()

                results.append({
                    "문제 ID": question_id,
                    "문제": question,
                    "답안 번호": i,
                    "학생 답안": student_answer,
                    "채점 기준": rubric,
                    "채점 결과": grading_result
                })

                print(f"[문제 {question_id} - 답안 {i}] 채점 완료")
                time.sleep(3)

        except Exception as e:
            print(f"[문제 {question_id}] 오류 발생: {e}")
            continue

# 전체 실행
generate_and_score_all()

# 엑셀 저장
df_result = pd.DataFrame(results)
df_result.to_excel("학생답안_채점결과_전체.xlsx", index=False)
print("엑셀 저장 완료: 학생답안_채점결과_전체.xlsx")


[문제 00041_42237 - 답안 1] 채점 완료
[문제 00041_42237 - 답안 2] 채점 완료
[문제 00041_42237 - 답안 3] 채점 완료
[문제 00041_42237 - 답안 4] 채점 완료
[문제 00041_42237 - 답안 5] 채점 완료
[문제 00041_41172 - 답안 1] 채점 완료
[문제 00041_41172 - 답안 2] 채점 완료
[문제 00041_41172 - 답안 3] 채점 완료
[문제 00041_41172 - 답안 4] 채점 완료
[문제 00041_41172 - 답안 5] 채점 완료
[문제 00041_41966 - 답안 1] 채점 완료
[문제 00041_41966 - 답안 2] 채점 완료
[문제 00041_41966 - 답안 3] 채점 완료
[문제 00041_41966 - 답안 4] 채점 완료
[문제 00041_41966 - 답안 5] 채점 완료
[문제 00041_42251 - 답안 1] 채점 완료
[문제 00041_42251 - 답안 2] 채점 완료
[문제 00041_42251 - 답안 3] 채점 완료
[문제 00041_42251 - 답안 4] 채점 완료
[문제 00041_42251 - 답안 5] 채점 완료
[문제 00041_42252 - 답안 1] 채점 완료
[문제 00041_42252 - 답안 2] 채점 완료
[문제 00041_42252 - 답안 3] 채점 완료
[문제 00041_42252 - 답안 4] 채점 완료
[문제 00041_42252 - 답안 5] 채점 완료
[문제 00041_42259 - 답안 1] 채점 완료
[문제 00041_42259 - 답안 2] 채점 완료
[문제 00041_42259 - 답안 3] 채점 완료
[문제 00041_42259 - 답안 4] 채점 완료
[문제 00041_42259 - 답안 5] 채점 완료
[문제 00041_35794 - 답안 1] 채점 완료
[문제 00041_35794 - 답안 2] 채점 완료
[문제 00041_35794 - 답안 3] 채점 완료
[문제 00041_